# Functional Programming for Data Analysis

### Jim Pivarski

First notebook: introduction

# x = x + 1

means **x** is infinite, right?

After a long day of messy details, it's good to remember what computers really are:


magical chalkboards that solve the problems we write on them.

<table style="width: 100%; height: 329px;">
<tr style="background-color: white;"><td><span style="font-family: Lato, sans-serif; font-size: 35.84px">Fun quiz: what were the first two high-level programming languages, both written for the IBM 704?</span></td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/7/7d/IBM_704_mainframe.gif" style="width: 500px; margin-left: auto; margin-right: auto;"></tr>
</table>

\#1 FORTRAN

\#2 Lisp

Although you might not think of it as a high-level language, the whole purpose of "FOR-TRAN" is to _translate formulas_ of human-readable mathematics into executable code.

<img src="http://worrydream.com/dbx/slides/slide.007.png" style="width: 600px; margin-left: auto; margin-right: auto;">

And Lisp is...

<img src="https://leanpub.com/site_images/lisphackers/lisp_cycles.png" style="width: 80%; margin-left: auto; margin-right: auto;">

In time, FORTRAN begat COBOL, ALGOL, Pascal, C, Objective C, C++, D, Rust...

("Practical languages for real work...")

While Lisp engendered Prolog, Scheme, ML, Haskell, Clojure...

("Conceptual languages that explore the bounds of computing...")

But lately, these two families have been converging: high-level languages are practical.

In the "multicore era," functional programming languages have been attracting attention as a way to simplify the human interface to parallel processing.

You specify less of the "how," more of the "what."

In [ ]:
map(lambda x: x**2, [1, 2, 3, 4, 5])

Did that just apply $f(x) = x^2$ to each integer from left to right, from right to left, or did it spawn five processes on five computers all around the world to produce the results and bring them back to me?

The abstraction, `map`, doesn't specify.

One popular application:

<table style="margin-left: auto; margin-right: auto;">
<tr style="background-color: white;"><td><img src="https://scr.sad.supinfo.com/articles/resources/207908/2807/1.png" style="height: 400px"></td>
<td><img src="https://pbs.twimg.com/profile_images/1252505253/elephant_rgb_sq.png" style="height: 200px"></tr>
</table>


The big deal about Hadoop was simply that a distributed shuffling system could be abstracted from the task in question by letting the users specify those tasks as function objects.

In [ ]:
def mapreduce(mapper, reducer, data):    # the lazy implementation; does not scale!
    output = {}
    for key1, value1 in data.items():
        for key2, value2 in mapper(value1):
            if key2 not in output:
                output[key2] = value2
            else:
                output[key2] = reducer(output[key2], value2)
    return output

In [ ]:
web_pages = {"one.html": "lol cat", "two.html": "justin bieber cat", "three.html": "bieber cat bieber"}

# every Hadoop tutorial starts with counting words on web pages
find_words = lambda page: [(word, 1) for word in page.split(" ")]
count_them = lambda count1, count2: count1 + count2

mapreduce(find_words, count_them, web_pages)

In [ ]:
residuals = {"track1": [0.10, -0.03, 0.07], "track2": [0.18, 0.04, -0.03], "track3": [-0.22, 0.13, -0.05]}

# but detector alignment is also a good application
collect_resids = lambda track: [("layer%d" % i, (1.0, track[i])) for i in range(3)]
average_them = lambda (w1, r1), (w2, r2): (w1 + w2, (w1*r1 + w2*r2)/(w1 + w2))

mapreduce(collect_resids, average_them, residuals)

Many problems fit into this same mold.

Yahoo, Facebook, LinkedIn, eBay, and IBM could all contribute to the same codebase and use it on their diverse problems, because the `mapreduce` function takes a function.

You've seen functions that take functions before:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/9f9f84b11bc8963b63f3775c2e268d8d9cde77e0)

The Lagrangian in physics.

"Functional," "functor," or "higher-order functions" are words for functions that take functions as arguments or return them as return values.

Functions are objects you can program with, just like any other object.

Functional languages have long been considered slow because the most natural implementation is to make the function some sort of class object to pass around.

However, some clever techniques can "compile away" the abstraction, so that you don't pay for it in performance.

**Another quiz:** what kind of language might this be?

```haskell
fun max(x: i32, y: i32): i32 =
  if x > y then x else y

fun redOp(x: (i32,i32,i32,i32))(y: (i32,i32,i32,i32)):
    (i32,i32,i32,i32) =
  let (mssx, misx, mcsx, tsx) = x in
  let (mssy, misy, mcsy, tsy) = y in
  (max(mssx, max(mssy, mcsx + misy)), max(misx, tsx+misy),
   max(mcsy, mcsx+tsy), tsx + tsy)

fun mapOp (x: i32): (i32,i32,i32,i32) =
  (max(x,0), max(x,0), max(x,0), x)

fun main(xs: []i32): i32 =
  let (x, _, _, _) =
    reduce redOp (0,0,0,0) (map mapOp xs) in x
```

**Answer:** it's [Futhark](https://futhark-lang.org/), a relative of Haskell specialized for GPU processing. It computes the maximum segment sum (e.g. `11` from `[1, -2, 3, 4, -1, 5, -6, 1]`), a non-commutative reducer, faster than Thrust (GPU library for C++).

<img src="https://futhark-lang.org/images/mss.svg" style="height: 400px; margin-left: auto; margin-right: auto;">

Functional programming languages are surprisingly well suited to designing FPGA circuits.

See [Bluespec](http://bluespec.com/) (closed source) and [CλaSH](http://www.clash-lang.org/) (open source).

Example: (a finite impulse response filter in CλaSH)

```haskell
fir coeffs x = dotp coeffs (window x)
  where
    dotp as bs = sum (zipWith (*) as bs)
```

Expressing a problem in terms of functions that take functions naturally lends itself to pipelining, unlike statements that change variable values.

And finally, it's used in data analysis because it hides computing details not relevant to the analyst and comes closer to the ideal of an executable chalkboard.

"R, at its heart, is a functional programming (FP) language." -- Hadley Wickham, [_Advanced R_](http://adv-r.had.co.nz/Functional-programming.html)

"Mathematica is Lisp with square brackets." -- me

Next notebook: [nb2-functional-playground.ipynb](nb2-functional-playground.ipynb)